In [1]:
from sage.all import *
import numpy as np
import sympy as sp
import random
import copy
from sympy.ntheory.modular import solve_congruence

In [2]:
np.random.seed(0)

In [3]:
class Constant:
    _N = 677
    _p = 512
    _q = 2048
    _pad_N = 1536
    
    _Y_coef = 3
    _Z_coef = 512
    
    _prime_q = 1419778561
    _root1536 = 11314797
    _root3 = 1025640636
    _root512 = 704288034
    
    def N(self):
        return Constant._N
    
    def p(self):
        return Constant._p
    
    def q(self):
        return Constant._q
    
    def pad_N(self):
        return Constant._pad_N
    
    def Y_coef(self):
        return Constant._Y_coef
    
    def Z_coef(self):
        return Constant._Z_coef
    
    def prime_q(self):
        return Constant._prime_q
    
    def root1536(self):
        return Constant._root1536
    
    def root3(self):
        return Constant._root3
    
    def root512(self):
        return Constant._root512

In [4]:
def show_matrix_2d(mtx):
    for i in range(len(mtx)):
        for j in range(len(mtx[i])):
            print(mtx[i][j], end=' ')
        print()
        
def show_matrix_1d(mtx):
    for i in mtx:
        print(i, end=' ')
    print()

In [5]:
pointwise_mul_sum_1d = lambda arr1, arr2: sum([arr1[i] * arr2[i] for i in range(len(arr1))])
pointwise_mul_1d = lambda arr1, arr2: [arr1[i] * arr2[i] for i in range(len(arr1))]
pointwise_mul_2d = lambda mtx1, mtx2: [[mtx1[i][j] * mtx2[i][j] for j in range(len(mtx1[0]))] for i in range(len(mtx1))]

In [6]:
constant = Constant()

In [7]:
Rx = PolynomialRing(IntegerModRing(constant.prime_q()), 'x')
Ry = PolynomialRing(IntegerModRing(constant.Y_coef()), 'y')
Rz = PolynomialRing(IntegerModRing(constant.Z_coef()), 'z')

a1 = [Rx(0) for _ in range((constant.Y_coef()*constant.Z_coef()))]
b1 = [[Rx(0) for _ in range(constant.Z_coef())] for _ in range(constant.Y_coef())]
a2 = [Rx(0) for _ in range((constant.Y_coef()*constant.Z_coef()))]
b2 = [[Rx(0) for _ in range(constant.Z_coef())] for _ in range(constant.Y_coef())]
school_book_a1 = None
school_book_a2 = None

for i in range(0, constant.N()):
    a1[i] = Rx(np.random.randint(256**4, dtype='<u4', size=1)[0]%constant.q())
    a2[i] = Rx(np.random.randint(256**4, dtype='<u4', size=1)[0]%constant.q())
    
school_book_a1 = copy.deepcopy(a1)
school_book_a2 = copy.deepcopy(a2)
school_book_a1 = [int(i.constant_coefficient()) for i in school_book_a1]
school_book_a2 = [int(i.constant_coefficient()) for i in school_book_a2]
    
school_book_mul = np.polymul(a1[::-1], a2[::-1])[::-1]

In [8]:
for i in range(len(a1)):
    b1[i%constant.Y_coef()][i%constant.Z_coef()] = a1[i]
    b2[i%constant.Y_coef()][i%constant.Z_coef()] = a2[i]

## 512(Z) axis ntt

In [9]:
#Turn Rx to int and throw into sympy
#int(Ry(i).constant_coefficient())
for i in range(constant.Y_coef()):
    b1[i] = [int(b1[i][j].constant_coefficient()) for j in range(constant.Z_coef())]
    b2[i] = [int(b2[i][j].constant_coefficient()) for j in range(constant.Z_coef())]
    b1[i] = sp.ntt(b1[i], constant.prime_q())
    b2[i] = sp.ntt(b2[i], constant.prime_q())

## 3(Y) axis NTT

In [10]:
radix3_ntt = [[Rx(1) for _ in range(3)] for _ in range(3)]

radix3_ntt[0][1] = Rx(constant.root3())
radix3_ntt[0][2] = Rx(power(constant.root3(), 2))

radix3_ntt[1][1] = Rx(power(constant.root3(), 2))
radix3_ntt[1][2] = Rx(power(constant.root3(), 4))

radix3_ntt[2][1] = Rx(power(constant.root3(), 3))
radix3_ntt[2][2] = Rx(power(constant.root3(), 3))

matrix_non_inverse = matrix(radix3_ntt)
matrix_inverse = matrix_non_inverse.inverse()
radix3_intt = [[matrix_inverse[i][j] for j in range(3)] for i in range(3)]

In [11]:
for j in range(constant.Z_coef()):
    b1[0][j], b1[1][j], b1[2][j] = \
    pointwise_mul_sum_1d([b1[0][j], b1[1][j],b1[2][j]], radix3_ntt[0]), \
    pointwise_mul_sum_1d([b1[0][j], b1[1][j],b1[2][j]], radix3_ntt[1]), \
    pointwise_mul_sum_1d([b1[0][j], b1[1][j],b1[2][j]], radix3_ntt[2])
    
    b2[0][j], b2[1][j], b2[2][j] = \
    pointwise_mul_sum_1d([b2[0][j], b2[1][j],b2[2][j]], radix3_ntt[0]), \
    pointwise_mul_sum_1d([b2[0][j], b2[1][j],b2[2][j]], radix3_ntt[1]), \
    pointwise_mul_sum_1d([b2[0][j], b2[1][j],b2[2][j]], radix3_ntt[2])

## Pointwise multiplication

In [12]:
b3 = pointwise_mul_2d(b1, b2)

## Inverse

In [13]:
#TODO make this part more readble
for j in range(constant.Z_coef()):
    b3[0][j], b3[1][j], b3[2][j] = \
    pointwise_mul_sum_1d([b3[0][j], b3[1][j],b3[2][j]], radix3_intt[0]), \
    pointwise_mul_sum_1d([b3[0][j], b3[1][j],b3[2][j]], radix3_intt[1]), \
    pointwise_mul_sum_1d([b3[0][j], b3[1][j],b3[2][j]], radix3_intt[2])

In [14]:
#Turn Rx to int and throw into sympy
for i in range(constant.Y_coef()):
    b3[i] = [int(b3[i][j]) for j in range(constant.Z_coef())]
    b3[i] = sp.intt(b3[i], constant.prime_q())

In [15]:
school_book_mul_3by512 = [[0 for _ in range(constant.Z_coef())] for _ in range(constant.Y_coef())]

school_book_mod2048 = [0 for _ in range(constant.Y_coef()*constant.Z_coef())]
for i in range(len(school_book_mul)):
    school_book_mul_3by512[i%constant.Y_coef()][i%constant.Z_coef()] = int(school_book_mul[i].constant_coefficient())

In [16]:
for i in range(constant.Y_coef()):
    for j in range(constant.Z_coef()):
        b3[i][j] = b3[i][j] % constant.q()
        school_book_mul_3by512[i][j] = school_book_mul_3by512[i][j] % constant.q()

In [17]:
a3 = [0 for _ in range(constant.Y_coef()*constant.Z_coef())]

for i in range(constant.Y_coef()):
    for j in range(constant.Z_coef()):
        a3[(1024*i + 513*j)%1536] = b3[i][j]

print() 
show_matrix_1d(a3)
print()
school_book_mul = np.polymul(a1[::-1], a2[::-1])[::-1]
school_book_mul = [int(i.constant_coefficient())%constant.q() for i in school_book_mul]
show_matrix_1d(school_book_mul)


1428 635 433 2040 1135 1224 1259 770 693 1802 598 1558 1790 1183 248 256 615 1587 1731 47 58 1573 1686 162 888 637 1304 1004 1935 1472 1811 706 133 1049 1711 909 1444 1524 1494 294 207 157 939 1649 1904 974 221 792 1604 622 611 1666 1362 564 472 1678 1628 1440 1786 30 537 310 75 1628 1804 1913 1794 1525 906 1143 1970 422 1338 1773 214 1886 796 202 936 399 284 841 409 1202 1303 1775 1714 998 136 787 1933 1252 1932 2032 1202 2046 1517 1754 1307 1501 1992 1721 1400 1674 1620 521 1916 621 1644 1980 314 1633 928 1849 625 203 510 1040 470 941 119 1271 1661 1203 1436 1518 473 822 890 1423 1827 623 1831 374 795 1953 1275 1083 1628 1141 1802 1383 786 1853 447 349 28 892 1234 1826 954 1608 1266 2015 1120 784 1759 1827 1794 1751 1143 1153 1794 909 1342 1496 654 1127 810 1752 1765 1869 597 1860 454 1367 1928 264 1768 237 1495 424 1344 1572 742 1893 1244 1457 2000 276 1209 1782 1507 606 1640 1740 799 1124 1569 631 1218 979 1427 1744 829 619 1527 1087 1549 393 1341 392 1693 1794 150 1674 1769 1760 